In [4]:
from pyspark.sql import SparkSession

In [3]:
#from pyspark.sql.functions import col, when, lit, sum, year, month, dayofmonth, dayofweek, dayofyear, weekofyear
#from pyspark.sql.types import IntegerType, FloatType, StringType, DateType
#from pyspark.sql.window import Window
#from pyspark.sql.types import DoubleType, FloatType, StringType
#from pyspark.sql.functions import year, countDistinct


In [5]:
# Crear una sesión de Spark
spark = SparkSession.builder \
    .appName("MiAplicacionSparkPostgreSQL") \
    .config("spark.jars", "/opt/spark/jars/postgresql-42.7.3.jar") \
    .config("spark.sql.debug.maxToStringFields", "100") \
    .getOrCreate()
    

# Configurar la conexión con PostgreSQL
jdbc_url = "jdbc:postgresql://localhost:5432/laboratorio"
connection_properties = {
    "user": "myuser",
    "password": "356241789",
    "driver": "org.postgresql.Driver"
}

24/08/24 22:25:53 WARN Utils: Your hostname, Angel resolves to a loopback address: 127.0.1.1; using 192.168.18.11 instead (on interface wlp3s0)
24/08/24 22:25:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/08/24 22:25:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [13]:
# Leer los archivos CSV en DataFrames de Spark
df_usuarios = spark.read.csv('Data/Usuarios.csv', header=True, inferSchema=True)
df_productos = spark.read.csv('Data/Productos.csv', header=True, inferSchema=True)
df_sucursales = spark.read.csv('Data/Sucursales.csv', header=True, inferSchema=True)
df_data = spark.read.csv('Data/Data.csv', header=True, inferSchema=True)

df_usuarios =df_usuarios.orderBy("nombre_completo", ascending=True)

# Configurar la conexión a PostgreSQL
jdbc_url = "jdbc:postgresql://localhost:5432/laboratorio"
db_properties = {
    "user": "myuser",
    "password": "356241789",  # Reemplaza con tu contraseña
    "driver": "org.postgresql.Driver"
}

# Cargar los datos en las tablas con mode='append'
df_usuarios.write.jdbc(url=jdbc_url, table="dim_usuarios", mode="append", properties=db_properties)


In [14]:
df_productos.write.jdbc(url=jdbc_url, table="dim_productos", mode="append", properties=db_properties)

In [15]:
df_sucursales.write.jdbc(url=jdbc_url, table="dim_sucursales", mode="append", properties=db_properties)


In [25]:
# Convertir la columna id_transaccion a entero
#df_data = df_data.withColumn("id_transaccion", col("id_transaccion").cast("int"))
df_data = df_data.withColumn("first_spend", lit(None).cast("boolean"))


In [ ]:
data_valid = df_data.join(df_usuarios, on='id_paciente', how='inner')

In [38]:

data_valid.write.jdbc(url=jdbc_url, table="data", mode="overwrite", properties=db_properties)

In [26]:
from pyspark.sql.functions import col

# Supongamos que tienes dos DataFrames: df_data y df_dim_usuarios

# Realiza una operación de left anti join para encontrar los registros en df_data que no están en df_dim_usuarios
faltantes = df_data.join(df_usuarios, on='id_paciente', how='left_anti')

# Cuenta el número de registros faltantes
cantidad_faltantes = faltantes.count()

print(f"Cantidad de id_paciente faltantes en dim_usuarios: {cantidad_faltantes}")

Cantidad de id_paciente faltantes en dim_usuarios: 6543


In [30]:
# Realiza un left anti join para encontrar los id_paciente en data que no están en dim_usuarios
pacientes_faltantes = df_data.join(df_usuarios, on='id_paciente', how='left_anti')

# Selecciona y muestra los id_paciente que faltan
pacientes_faltantes.select("id_paciente").distinct().show()


+-----------+
|id_paciente|
+-----------+
| 1044640433|
| 1043715813|
|    5487444|
| 1082047266|
| 1085008315|
|   73226526|
|   32689530|
| 1140875690|
| 1082046030|
|    8699759|
|    6202054|
|   88261075|
|   22730682|
| 1048221639|
| 1143163962|
|   22483113|
|    1404966|
|    3945074|
| 1151480952|
| 1140891461|
+-----------+
only showing top 20 rows



In [34]:
from pyspark.sql import functions as F
# Realiza el conteo de transacciones para cada id_paciente en pacientes_faltantes
conteo_transacciones_por_paciente = pacientes_faltantes.groupBy("id_paciente").agg(F.count("id_transaccion").alias("total_transacciones"))

# Muestra los resultados
conteo_transacciones_por_paciente.show()

+-----------+-------------------+
|id_paciente|total_transacciones|
+-----------+-------------------+
| 1044640433|                  1|
| 1043715813|                  2|
|    5487444|                  3|
| 1082047266|                  4|
| 1085008315|                  5|
|   73226526|                  3|
|   32689530|                  1|
| 1140875690|                  3|
| 1082046030|                  3|
|    8699759|                  1|
|    6202054|                  1|
|   88261075|                  2|
|   22730682|                  4|
| 1048221639|                  2|
| 1143163962|                  3|
|   22483113|                  3|
|    1404966|                  4|
|    3945074|                 11|
| 1151480952|                  2|
| 1140891461|                  4|
+-----------+-------------------+
only showing top 20 rows

